##### Here we will do all the data exploration which are more bitcoin specific and requires more knowledge about bitcoin ->


In [1]:
import os
import json
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Work.Helper import list_all_tx  # Import the function from Script.py
from Work.Script import *
# import Opcodes
# from Work.Helper import little_endian_to_int,list_all_tx,int_to_little_endian

all_files= list_all_tx() 

In [2]:
import sys
sys.path

['/home/divyanshgupta/code-challenge-2024-15IITian/Work/Data_Enginerring',
 '/usr/lib64/python311.zip',
 '/usr/lib64/python3.11',
 '/usr/lib64/python3.11/lib-dynload',
 '',
 '/home/divyanshgupta/code-challenge-2024-15IITian/.venv/lib64/python3.11/site-packages',
 '/home/divyanshgupta/code-challenge-2024-15IITian/.venv/lib/python3.11/site-packages',
 '/home/divyanshgupta/code-challenge-2024-15IITian',
 '/home/divyanshgupta/code-challenge-2024-15IITian/Work']

## Analysis of txns including `P2SH`->


In [3]:
witness_empty=0
witness_empty_file=[]
redeem_witness_empty=[]
redeem_empty_witness_present=[]
redeem_present_witness_present=0
psh=0
redeem_present_witness_empty= []

for file in all_files:
   
        for input in file['vin']:
         
            if input['prevout']['scriptpubkey_type']== 'p2sh':
               psh+=1
               if input.get("witness")==None:
                    witness_empty+=1
                    witness_empty_file.append(file)
                    if input.get("inner_redeemscript_asm")==None:
                         redeem_witness_empty.append(file)
                         
               if input.get("witness")==None:
                    witness_empty+=1
                    witness_empty_file.append(file)
               if input.get("witness") != None and input.get("inner_redeemscript_asm")==None:
                    redeem_empty_witness_present.append(file)

               if input.get("witness") != None: 
                  if input.get("inner_redeemscript_asm")!=None:
                     redeem_present_witness_present+=1   
               if input.get("witness") == None and input.get("inner_redeemscript_asm")==None:
                    redeem_present_witness_empty.append(file)
                      


# print(witness_empty)                    
                   
# print(witness_empty_file[0])
print(redeem_witness_empty)

print(redeem_empty_witness_present)
print(redeem_present_witness_present)
print(psh)
print(len(redeem_present_witness_empty))


              


[]
[]
0
38
0


##### Observation ->

- If in all tx script type -> there is no mention of `p2sh-p2wkh` and `p2sh-p2wsh` script type.
- Note : `P2sh` -> Total = 3672 , `p2sh` = 38 , nested_p2sh = rest

### Idea to segreate them ->

- `p2sh` -> no witness + yes(inner_redeem_script)
- `p2sh-p2wpkh` -> witness + inner_redeem_script - inner_witness_script
- `p2sh-p2wsh` -> witness + inner_redeem_script + inner_witness_script
- **Note** -> We will change only in vin as in case of vout -> we don't know any more info from the script_pubkey alone


In [4]:
folder_path = "../../mempool"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    
        for input in data["vin"]: 
            script_type = input["prevout"]["scriptpubkey_type"]
            if script_type == "p2sh":
                  if input.get("witness") != None and input.get("inner_redeemscript_asm") != None:
                      if input.get("inner_witnessscript_asm") != None:
                         input["prevout"]["scriptpubkey_type"] ="v0_p2sh_p2wsh"
                      else:
                          input["prevout"]["scriptpubkey_type"]="v0_p2sh_p2wpkh"
        with open(file_path, 'w') as f:
          json.dump(data, f, indent=4)
                            

**Check** witness field should not exist and script_sig not empty for legacy address and vice versa for segwit script


- Check whether `parse(sript_pubkey)` == `scriptpubkey_asm` or not ?
- `parse(sript_sig)` == `scriptsig_asm`?
- Find all opcodes used in the script for definign the function
-


In [5]:
script_pubkey_unmatch=[]
script_sig_unmatch=[]
all_opcodes_used = set() 
for file_index, file in enumerate(all_files):
    for section in ["vout","vin"]:
        for input_index, input in enumerate(file[section]):
            scriptpubkey_asm=""
            parsed_script_pubkey=""
            parsed_script_sig=""
            scriptsig_asm=""
            if section == "vin":
                parsed_script_pubkey=  Script.parse(input["prevout"]["scriptpubkey"])
                parsed_script_sig= Script.parse(input["scriptsig"])
                scriptpubkey_asm= input["prevout"]["scriptpubkey_asm"]
                scriptsig_asm= input["scriptsig_asm"]
            else:
                parsed_script_pubkey= Script.parse(input["scriptpubkey"])
                scriptpubkey_asm= input["scriptpubkey_asm"]

            if str(parsed_script_pubkey)!= scriptpubkey_asm:
                script_pubkey_unmatch.append([file_index,section])    
            if str(parsed_script_sig) !=  scriptsig_asm:
                script_sig_unmatch.append([file_index,section,input_index]) 
            # finding all the opcodes used in the script ->     
            for script_type in [scriptpubkey_asm,scriptsig_asm]:
                for word in script_type.split():
                    if word[:2] == "OP":
                        all_opcodes_used.add(word)




print(len(script_pubkey_unmatch))
print("script_pubkey_unmatch : {}".format(script_pubkey_unmatch))
print("script_sig_unmatch:{}".format(script_sig_unmatch))  
print("all opcodes used : {}".format(all_opcodes_used))              

                


0
script_pubkey_unmatch : []
script_sig_unmatch:[]
all opcodes used : {'OP_PUSHBYTES_17', 'OP_PUSHBYTES_32', 'OP_PUSHBYTES_20', 'OP_PUSHBYTES_69', 'OP_PUSHBYTES_42', 'OP_PUSHBYTES_72', 'OP_RETURN', 'OP_0', 'OP_PUSHDATA2', 'OP_PUSHBYTES_46', 'OP_PUSHNUM_3', 'OP_PUSHBYTES_73', 'OP_DUP', 'OP_CHECKSIG', 'OP_PUSHBYTES_33', 'OP_HASH160', 'OP_PUSHBYTES_4', 'OP_PUSHBYTES_70', 'OP_PUSHBYTES_31', 'OP_PUSHBYTES_71', 'OP_PUSHBYTES_54', 'OP_PUSHBYTES_9', 'OP_PUSHBYTES_64', 'OP_EQUALVERIFY', 'OP_CHECKMULTISIG', 'OP_PUSHBYTES_12', 'OP_PUSHBYTES_5', 'OP_PUSHNUM_1', 'OP_PUSHBYTES_74', 'OP_PUSHBYTES_10', 'OP_PUSHBYTES_34', 'OP_PUSHBYTES_67', 'OP_PUSHBYTES_22', 'OP_PUSHBYTES_1', 'OP_PUSHBYTES_65', 'OP_PUSHBYTES_68', 'OP_PUSHBYTES_13', 'OP_PUSHDATA1', 'OP_PUSHBYTES_11', 'OP_EQUAL', 'OP_PUSHBYTES_63'}


#### Observation ->

- `Scriptpubkey` matches with `Scriptpubkey_asm` + script_case for all txn
- Total 41 opcodes are used -> `all_opcodes_used`


##### Check whether if the scriptpubkey type is legacy then it should not contain witness field and segwit script should contain this field

##### Check `OP_RETURN` scripttype should come where `value = 0`


##### Check scriptpubkey corresponds to scriptpubkey_type:


##### Check if for any tx -> fee <= 0 -> if so then tx is invalid


In [9]:
legacy_contain_witness_vice_versa=[]
op_return_then_zero_value=[]
op_return_no=0

# scripttype with scriptpubkey
scriptpub_matches_with_scriptpubtype=set()

# collect all tx whose fee<= 0
invalid_tx_with_fee_lessthan_zero=[]
for file in all_files:
    for section in ["vin","vout"]:
     if section == "vin":
      for input in file["vin"]:
        script_type= input["prevout"]["scriptpubkey_type"]
        scriptpubkey_asm= input["prevout"]["scriptpubkey_asm"]
        ans=  Script.is_scriptpubkey_match_scripttype( scriptpubkey_asm,script_type)
        scriptpub_matches_with_scriptpubtype.add(ans)


        if input["prevout"]["scriptpubkey_type"] in ["p2sh","p2pkh","bare-multisig"]:
            if input.get("witness")!=None:
                legacy_contain_witness_vice_versa.append(file)
        else:
            if  input.get("witness") == None:
                legacy_contain_witness_vice_versa.append(file) 
     else:
        for output in file["vout"]:
           if output["scriptpubkey_type"]=="op_return":
              op_return_no+=1
              if output["value"]==0:
                 op_return_then_zero_value.append(file)
              
           
        
                
                   
print(legacy_contain_witness_vice_versa)
print(op_return_no)
print(len(op_return_then_zero_value))
scriptpub_matches_with_scriptpubtype

[]
117
117


{True}

**Observation :**

- legacy script not have witness & segwit script have witness field -> all ok
- Wherever scripttype is `op_return` -> value =0
- For every tx -> given scriptpubkey is corresponding to given script_type -> all ok
